In [13]:
import os.path
import shutil

import numpy as np
import pandas as pd

df = pd.read_csv("data resources/sGSL-Northumberland-Survey-FGP.csv", encoding="windows-1252")
df

,cruise_number__numero_de_criosiere,year__annee,month__mois,day_jour,start_hour__heure_de_depart,start_minute__minute_de_depart,latitude,longitude,gear__equipement,species__espece,french_name__nom_francais,english_name__nom_anglais,latin_name__nom_latin,weight_caught__poids_pris,number_caught__quantite__attrape
0,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,43,Plie rouge,Winter flounder,Pseudopleuronectes americanus,0.500,1.0
1,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,122,Tanche-tautogue,Cunner,Tautogolabrus adspersus,0.800,2.0
2,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,2550,Homard américain,American lobster,Homarus americanus,3.800,16.0
3,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,4330,Moule non-spécifié,Mussel unidentified,Mytilidae (f.),0.010,1.0
4,24,2000,7,25,10.0,39.0,46.774167,-64.750917,No. 286 otter trawl with rockhopper,6200,Ophiuroidea non-spécifié,Brittle star unidentified,Ophiuroidea (s.o.),0.010,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27873,151,2021,8,2,10.0,46.0,45.963083,-63.379667,Northumberland trawl,2417,Crevette grise,Sevenspine bay shrimp,Crangon septemspinosa,0.001,0.0
27874,151,2021,8,2,10.0,46.0,45.963083,-63.379667,Northumberland trawl,2513,Crabe commun,Atlantic rock crab,Cancer irroratus,0.634,3.0
27875,151,2021,8,2,10.0,46.0,45.963083,-63.379667,Northumberland trawl,2550,Homard américain,American lobster,Homarus americanus,2.980,4.0
27876,151,2021,8,2,10.0,46.0,45.963083,-63.379667,Northumberland trawl,4511,Encornet rouge nordique,Northern shortfin squid,Illex illecebrosus,0.102,1.0


In [14]:
# define the function to combine datetime from a row
def combine_date(row):
    year = str(row["year__annee"]).zfill(4)
    month = str(row["month__mois"]).zfill(2)
    day = str(row["day_jour"]).zfill(2)
    hour = row["start_hour__heure_de_depart"]
    minute = row["start_minute__minute_de_depart"]
    if (not isinstance(hour, str) and np.isnan(hour)) or (not isinstance(minute, str) and np.isnan(minute)):
        return f"{year}-{month}-{day}"
    else:
        if int(minute) > 59:
            minute = 59
        return f"{year}-{month}-{day} {int(hour)}:{int(minute)}"

In [15]:
# add a new column that is a datetime
df["datetime"] = pd.to_datetime(df.apply(combine_date, axis=1))
df_trunc = df.sort_values("datetime").loc[:, ["year__annee", "latitude", "longitude", "datetime"]]
df_trunc = df_trunc.drop_duplicates()
df_trunc

,year__annee,latitude,longitude,datetime
0,2000,46.774167,-64.750917,2000-07-25 10:39:00
6,2000,46.803500,-64.804250,2000-07-25 11:27:00
12,2000,46.838167,-64.837333,2000-07-25 12:40:00
13,2000,46.869500,-64.840333,2000-07-25 20:29:00
23,2000,46.895083,-64.795417,2000-07-26 08:06:00
...,...,...,...,...
27829,2021,45.900667,-63.285667,2021-08-01 17:05:00
27844,2021,45.832167,-63.190417,2021-08-02 07:00:00
27852,2021,45.865083,-63.327167,2021-08-02 08:25:00
27864,2021,45.898333,-63.427917,2021-08-02 09:34:00


In [16]:
from stock_assessment_surveys.utils.xy2line import xy2line
xy2line(df_trunc)